# CLFO
## 01 Data preparation and selection

In [ ]:
import os
from functions import *
import pandas as pd
import numpy as np

### Image labels (level clasification)

In [ ]:
outputs = pd.read_csv("d:/project_data/trainLabels.csv")

### Image paths

In [ ]:
paths = ["d:/project_data/train_1","d:/project_data/train_2","d:/project_data/train_3","d:/project_data/train_4"]

### Inspecting level ratio for each dateset

In [ ]:
for path in paths:
    image_files=os.listdir(path)
    data = selector(image_files,outputs)
    print("\n",path)
    print(data["level"].value_counts())   
    print(data['level'].iloc[0])

### Selecting 6000 samples (1200 for each level)

In [ ]:
num_classes = 5
img_paths = [ [] for _ in range (num_classes)]

proceed = 5
level_status = [0,0,0,0,0]

while proceed:
    for path in paths:
        image_files=os.listdir(path)
        data = selector(image_files, outputs)
        print("\n",path)
        print(data["level"].value_counts())
        print(len(image_files))
        for row in range(len(image_files)):
            level = int(data["level"].iloc[row])
            #print(level, end=' ')
            if level_status[level] < 1200:
                img_paths[level].append(os.path.join(path, image_files[row]))
            elif level_status[level] == 1200:
                proceed-=1
            level_status[level]+=1         
        print(level_status)

In [ ]:
for i in range(5):
    print(f'Number of samples for category {i}: {len(img_paths[i])}')

### Storing data as numpy matrix

#### Images are transformed into B&W and resized to 256 x 256

In [ ]:
# takes too long - do not load

n_samples = 6000
X = np.zeros((n_samples,256,256))
Y = np.zeros(n_samples)

index = 0
for i in range(num_classes):
    for path in img_paths[i]:
        img = load(path)
        img = resize(img)
        #img = masking(img) #filter most common color #it will take 20h
        img = grey(img)
        img_resized = model_resize(img)
        X[index] = img_resized
        Y[index] = i
        index+=1
        if index%100 == 0:
            print(f"{index}/{n_samples}")
            
print('End of data storing')

### Saving data in .npy file

In [ ]:
with open("tr2_mask_256.npy","wb")as f:
    np.save(f,X)
    np.save(f,Y)